In [19]:
import pandas as pd
import numpy as np

用online pred 的结果代替销量周销量0-15的sku+dc的预测结果，看mape是否下降

In [20]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [21]:
raw = pd.read_csv('./result_670/raw_pred_result_7371_0508-step28.csv')

In [22]:
new = pd.read_csv('./result_670/transfer_test_result_7371_0508-step28.csv')

In [23]:
online = pd.read_csv('./result_670/7371_pred_20180508.csv',sep='\t').rename(columns={'dt':'Date','dcid':'RDCKey','wid':'ProductKey'})

In [25]:
online.drop(online.columns[0],axis=1, inplace=True)
online = online[['Date', 'RDCKey','ProductKey','model','ofdsales']]

In [26]:
print(raw.shape)
print(new.shape)
print(online.shape)

(601468, 137)
(601468, 8)
(204834, 5)


In [27]:
raw.head()

,Date,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,OrderNonOutlierVolume,ProductKey,RDCKey,brand_code,dt_onstock_min,reserveState,salesForecast_x,vendibility1,skuDecomposedSeasonal,skuDecomposedTrend,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,dt,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,SyntheticPromotionSubType,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,ypred,sales,salesForecast_y
0,2018-05-08,670.0,729.0,7371.0,1.0,247046,3.0,15097.0,2017-04-04,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,5,1,0.0,0.145714,0.145714,0.0,0.08,0.0,0.145714,0.145714,0.0,0.08,0.0,0.145714,0.145714,0.0,0.08,1.0,32.5,0.0,NaN,NaN,NaN,2018-05-08,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,22.27,0.0,NaN,NaN,NaN,2.857143,3.142857,3.8,4.0,5.5,9.0,2.0,2.0,3.987491,3.547352,3.965761,5.590022,1.675,1.6,0.988889,1.035714,0.857143,0.857143,1.0,0.333333,0.5,1.0,1.0,1.0,1.517748,1.351589,1.122086,0.939241,0.875386,0.820443,0.739828,0.0,1.642857,1.142857,1.0,0.333333,0.5,1.0,1.5,1.0,2.117266,1.596996,1.073355,0.761157,0.0,6.0,0.0,0.0,0.0,29.0,29.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.171429,0.107692,0.171429,0.0,29.0,29.0,29.0,32.5,0.0,0.1077,0.0,0.1077,1.178358,0,0
1,2018-05-09,670.0,729.0,7371.0,1.0,247046,3.0,15097.0,2017-04-04,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,5,2,0.0,0.145714,0.145714,0.0,0.08,0.0,0.145714,0.145714,0.0,0.08,0.0,0.145714,0.145714,0.0,0.08,1.0,32.5,0.0,NaN,NaN,NaN,2018-05-09,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,22.27,0.0,NaN,NaN,NaN,2.857143,3.142857,3.8,4.0,5.5,9.0,2.0,2.0,3.987491,3.547352,3.965761,5.590022,1.675,1.6,0.988889,1.035714,0.857143,0.857143,1.0,0.333333,0.5,1.0,1.0,1.0,1.517748,1.351589,1.122086,0.939241,0.875386,0.820443,0.739828,0.0,1.642857,1.142857,1.0,0.333333,0.5,1.0,1.5,1.0,2.117266,1.596996,1.073355,0.761157,0.0,6.0,0.0,0.0,0.0,29.0,29.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.171429,0.107692,0.171429,0.0,29.0,29.0,29.0,32.5,0.0,0.1077,0.0,0.1077,1.121724,3,3
2,2018-05-10,670.0,729.0,7371.0,1.0,247046,3.0,15097.0,2017-04-04,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,3,0,0,0,0,0

In [28]:
new.head()

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ensemble
0,2018-05-08,3.0,247046,1.043977,1.009120,0,0,1.026252
1,2018-05-09,3.0,247046,0.991574,1.078592,3,3,1.033254
2,2018-05-10,3.0,247046,0.943584,1.006736,0,0,0.974137
3,2018-05-11,3.0,247046,0.867182,0.964858,2,2,0.913416
4,2018-05-12,3.0,247046,0.723614,0.740671,0,0,0.732043


In [29]:
online.head()

,Date,RDCKey,ProductKey,model,ofdsales
0,2018-05-08,6,1024672,lr_minrdc_model_guan,0.114541
1,2018-05-09,6,1024672,lr_minrdc_model_guan,0.114541
2,2018-05-10,6,1024672,lr_minrdc_model_guan,0.114541
3,2018-05-11,6,1024672,lr_minrdc_model_guan,0.114541
4,2018-05-12,6,1024672,lr_minrdc_model_guan,0.114541


In [30]:
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204834 entries, 0 to 204833
Data columns (total 5 columns):
Date          204834 non-null object
RDCKey        204834 non-null int64
ProductKey    204834 non-null int64
model         204834 non-null object
ofdsales      204834 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.8+ MB


In [31]:
list_keys = ['Date','RDCKey','ProductKey']
raw = raw[list_keys + ['ypred','salesForecast_y']].rename(columns={"ypred":"ypred_raw"})

In [32]:
raw.Date = pd.to_datetime(raw.Date)
new.Date = pd.to_datetime(new.Date)
online.Date = pd.to_datetime(online.Date)

In [33]:
df = pd.merge(new, raw, how='left', on=list_keys)

In [34]:
df = df.merge(online, how='left', on=list_keys)

In [35]:
df

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ensemble,ypred_raw,salesForecast_y,model,ofdsales
0,2018-05-08,3,247046,1.043977,1.009120,0,0,1.026252,1.178358,0,lr_model_id,1.049928
1,2018-05-09,3,247046,0.991574,1.078592,3,3,1.033254,1.121724,3,lr_model_id,0.935206
2,2018-05-10,3,247046,0.943584,1.006736,0,0,0.974137,1.121724,0,lr_model_id,1.197808
3,2018-05-11,3,247046,0.867182,0.964858,2,2,0.913416,1.043352,2,lr_model_id,0.973876
4,2018-05-12,3,247046,0.723614,0.740671,0,0,0.732043,0.919101,0,lr_model_id,0.609102
5,2018-05-13,3,247046,0.723614,0.646271,0,0,0.682759,0.877043,0,lr_model_id,0.608138
6,2018-05-14,3,247046,1.295287,1.283237,5,5,1.289234,1.344315,5,lr_model_id,1.079096
7,2018-05-15,3,247046,1.295287,1.269854,1,1,1.282444,1.344315,1,NaN,NaN
8,2018-05-16,3,247046,0.991574,1.129142,8,8,1.055896,1.121724,8,NaN,NaN
9,2018-05-17,3,247046,0.943584,1.032422,0,0,0.986006,1.121724,0,NaN,NaN


In [36]:
df2 = df[df.Date<='2018-06-04']
df2['final_ensemble'] = 2 / ( 1 / df2.ypred_raw + 1 / df2.model_2_ypred)

df2['final_ensemble_2'] = (df2.ypred_raw + df2.model_2_ypred) / 2

#df2['final_ensemble_3'] = (0.303/(0.294+0.303)) * df2.ypred_raw + (0.294/(0.294+0.303)) * df2.model_2_ypred

print(df2.final_ensemble.sum())
print(np.sum(np.abs(df2.final_ensemble - df2.salesForecast)) / np.sum(df2.salesForecast))

print(df2.final_ensemble_2.sum())
print(np.sum(np.abs(df2.final_ensemble_2 - df2.salesForecast)) / np.sum(df2.salesForecast))

#print df2.final_ensemble_3.sum()
#print np.sum(np.abs(df2.final_ensemble_3 - df2.salesForecast)) / np.sum(df2.salesForecast)

print(df2.salesForecast.sum())
print(df2.ypred_raw.sum())
print(df2.model_2_ypred.sum())
print("\n")
print(np.sum(np.abs(df2.ypred_raw - df2.salesForecast)) / np.sum(df2.salesForecast))
print(np.sum(np.abs(df2.model_2_ypred - df2.salesForecast)) / np.sum(df2.salesForecast))


909380.414177
0.765740001074
955548.360177
0.783927648772
990036
923569.154203
987527.566152


0.779840189669
0.81757368284


In [39]:
df2.head()

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ensemble,ypred_raw,salesForecast_y,model,ofdsales,final_ensemble,final_ensemble_2
0,2018-05-08,3,247046,1.043977,1.009120,0,0,1.026252,1.178358,0,lr_model_id,1.049928,1.087192,1.093739
1,2018-05-09,3,247046,0.991574,1.078592,3,3,1.033254,1.121724,3,lr_model_id,0.935206,1.099735,1.100158
2,2018-05-10,3,247046,0.943584,1.006736,0,0,0.974137,1.121724,0,lr_model_id,1.197808,1.061124,1.064230
3,2018-05-11,3,247046,0.867182,0.964858,2,2,0.913416,1.043352,2,lr_model_id,0.973876,1.002571,1.004105
4,2018-05-12,3,247046,0.723614,0.740671,0,0,0.732043,0.919101,0,lr_model_id,0.609102,0.820295,0.829886


In [38]:
df22 = df2[(df2.Date<='2018-05-14')]
df3 = df22.groupby(['RDCKey','ProductKey']).sum().reset_index()
#df3['final_ensemble'] = 2 / ( 1 / df3.ypred_raw + 1 / df3.model_2_ypred)

#df3['final_ensemble_2'] = (df3.ypred_raw + df3.model_2_ypred) / 2

#df3['final_ensemble_3'] = (0.303/(0.294+0.303)) * df2.ypred_raw + (0.294/(0.294+0.303)) * df2.model_2_ypred

print (df3.final_ensemble.sum())
print (np.sum(np.abs(df3.final_ensemble - df3.salesForecast)) / np.sum(df3.salesForecast))

print (df3.final_ensemble_2.sum())
print (np.sum(np.abs(df3.final_ensemble_2 - df3.salesForecast)) / np.sum(df3.salesForecast))

#print df2.final_ensemble_3.sum()
#print np.sum(np.abs(df2.final_ensemble_3 - df2.salesForecast)) / np.sum(df2.salesForecast)
print ("\n")
print (df3.salesForecast.sum())
print (df3.ypred_raw.sum())
print (df3.model_2_ypred.sum())
print ("\n")
print (np.sum(np.abs(df3.ypred_raw - df3.salesForecast)) / np.sum(df3.salesForecast))
print (np.sum(np.abs(df3.model_2_ypred - df3.salesForecast)) / np.sum(df3.salesForecast))



229806.877206
0.393911769101
240333.815627
0.400606393236


251373
228522.228281
252145.402972


0.404166073332
0.423871288865


In [42]:
df3.head(10)

,RDCKey,ProductKey,model_2_ypred,ensemble,ypred_raw,salesForecast_y,ofdsales,final_ensemble,final_ensemble_2
0,3.0,247046,6.729485,6.651096,7.605618,10,6.453155,7.128161,7.167552
1,3.0,247048,1.712392,1.615216,1.499655,1,2.755000,1.582146,1.606023
2,3.0,385604,1.933674,0.239931,1.233213,0,NaN,0.664163,1.583443
3,3.0,385612,302.197403,238.895996,279.539525,210,225.348452,281.331853,290.868464
4,3.0,385627,87.708409,89.256198,85.347943,60,76.682175,84.629268,86.528176
5,3.0,385632,30.696890,29.095059,29.409510,11,41.202045,29.239728,30.053200
6,3.0,385643,26.499378,23.573822,24.672318,18,31.726119,25.347361,25.585848
7,3.0,385649,23.418613,23.572190,22.459945,30,20.836543,22.827287,22.939279
8,3.0,385870,53.809448,57.373194,48.713553,130,73.785589,50.318449,51.261500
9,3.0,385874,216.267107,166.718498,156.625461,290,159.705894,174.259594,186.446284


In [41]:
df3.drop(['ypred','sales','salesForecast'],axis=1,inplace=True)

In [43]:
df3['model2_pred_mape'] = np.abs(df3.model_2_ypred - df3.salesForecast_y) / df3.salesForecast_y
df3['raw_pred_mape'] = np.abs(df3.ypred_raw - df3.salesForecast_y) / df3.salesForecast_y
df3['online_pred_mape'] = np.abs(df3.ofdsales - df3.salesForecast_y) / df3.salesForecast_y
df3['ensemble_mape'] = np.abs(df3.final_ensemble - df3.salesForecast_y) / df3.salesForecast_y

In [45]:
print (df3[df3.raw_pred_mape==np.inf].shape)
print (df3[df3.salesForecast_y==0].shape)

(10516, 13)
(10517, 13)


In [49]:
df4 = df3[df3.salesForecast_y!=0]

In [60]:
(df4.salesForecast_y == 0).any()

False

In [68]:
np.geterr()

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [71]:
cate3 = 7371
sales_threshold = np.arange(0, 2000, 100)
tmp_list = []
for i in range(len(sales_threshold)):
    if i != len(sales_threshold)-1:
        tmp = df4[(sales_threshold[i]<df4.salesForecast_y)&(df4.salesForecast_y <= sales_threshold[i+1])]
        print('range in ',sales_threshold[i], sales_threshold[i+1], 'shape:', tmp.shape)
#         print(type(np.sum(tmp.salesForecast_y)))
#         print(type(np.sum(np.abs(tmp.ypred_raw - tmp.salesForecast_y))))
        tmp_oneline_df = pd.DataFrame()
        tmp_oneline_df['cate3'] = [cate3]
        tmp_oneline_df['range_min'] = [sales_threshold[i]]
        tmp_oneline_df['range_max'] = [sales_threshold[i+1]]
        tmp_oneline_df['count_num'] = [tmp.shape[0]]
        tmp_oneline_df['sales_ratio_total'] = [float(tmp.salesForecast_y.sum()) / df4.salesForecast_y.sum()]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['online_pred_sum'] = [tmp.ofdsales.sum()]
        tmp_oneline_df['ypred_raw_sum'] = [tmp.ypred_raw.sum()]
        tmp_oneline_df['model_2_ypred_sum'] = [tmp.model_2_ypred.sum()]
        tmp_oneline_df['ensemble_sum'] = [tmp.final_ensemble.sum()]
        tmp_oneline_df['actual_sales_sum'] = [tmp.salesForecast_y.sum()]
        tmp_oneline_df['online_pred_mape'] = [np.sum(np.abs(tmp.ofdsales - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ypred_raw_mape'] = [np.sum(np.abs(tmp.ypred_raw - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['model_2_ypred_mape'] = [np.sum(np.abs(tmp.model_2_ypred - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ensemble_mape'] = [np.sum(np.abs(tmp.final_ensemble - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
    else:
        print('larger than',sales_threshold[i], 'shape:', tmp.shape)
        tmp = df4[(sales_threshold[i]<df4.salesForecast_y)]
        tmp_oneline_df = pd.DataFrame()
        tmp_oneline_df['cate3'] = [cate3]
        tmp_oneline_df['range_min'] = [sales_threshold[i]]
        tmp_oneline_df['range_max'] = [-999]
        tmp_oneline_df['count_num'] = [tmp.shape[0]]
        tmp_oneline_df['sales_ratio_total'] = [float(tmp.salesForecast_y.sum()) / df4.salesForecast_y.sum()]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['online_pred_sum'] = [tmp.ofdsales.sum()]
        tmp_oneline_df['ypred_raw_sum'] = [tmp.ypred_raw.sum()]
        tmp_oneline_df['model_2_ypred_sum'] = [tmp.model_2_ypred.sum()]
        tmp_oneline_df['ensemble_sum'] = [tmp.final_ensemble.sum()]
        tmp_oneline_df['actual_sales_sum'] = [tmp.salesForecast_y.sum()]
        tmp_oneline_df['online_pred_mape'] = [np.sum(np.abs(tmp.ofdsales - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ypred_raw_mape'] = [np.sum(np.abs(tmp.ypred_raw - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['model_2_ypred_mape'] = [np.sum(np.abs(tmp.model_2_ypred - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ensemble_mape'] = [np.sum(np.abs(tmp.final_ensemble - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
    tmp_list.append(tmp_oneline_df)
        
stat_df = pd.concat(tmp_list)          


range in  0 100 shape: (10455, 13)
range in  100 200 shape: (281, 13)
range in  200 300 shape: (99, 13)
range in  300 400 shape: (43, 13)
range in  400 500 shape: (29, 13)
range in  500 600 shape: (17, 13)
range in  600 700 shape: (5, 13)
range in  700 800 shape: (8, 13)
range in  800 900 shape: (3, 13)
range in  900 1000 shape: (7, 13)
range in  1000 1100 shape: (9, 13)
range in  1100 1200 shape: (0, 13)
range in  1200 1300 shape: (2, 13)
range in  1300 1400 shape: (2, 13)
range in  1400 1500 shape: (1, 13)
range in  1500 1600 shape: (0, 13)
range in  1600 1700 shape: (0, 13)
range in  1700 1800 shape: (0, 13)
range in  1800 1900 shape: (0, 13)
larger than 1900 shape: (0, 13)


In [72]:
stat_df

,cate3,range_min,range_max,count_num,sales_ratio_total,online_pred_sum,ypred_raw_sum,model_2_ypred_sum,ensemble_sum,actual_sales_sum,online_pred_mape,ypred_raw_mape,model_2_ypred_mape,ensemble_mape
0,7371,0,100,10455,0.431307,97206.174191,95690.320522,111824.364359,98499.383458,108419,0.569439,0.536630,0.575582,0.526516
0,7371,100,200,281,0.158350,31058.225495,32652.342700,33077.621963,31451.500277,39805,0.405612,0.366024,0.335821,0.345650
0,7371,200,300,99,0.096347,19073.434082,20260.755641,20523.055582,19495.164874,24219,0.325025,0.288833,0.268603,0.277840
0,7371,300,400,43,0.058825,12789.637921,13047.632897,13218.379653,12724.333312,14787,0.360351,0.257033,0.271359,0.263459
0,7371,400,500,29,0.050117,9750.888907,9798.110316,10936.852866,9990.924870,12598,0.306278,0.276712,0.244825,0.262866
0,7371,500,600,17,0.037864,7941.658472,8343.052314,8792.918352,8350.492285,9518,0.239952,0.218435,0.165455,0.190085
0,7371,600,700,5,0.012861,2618.870487,2949.449769,2882.389237,2823.604639,3233,0.215792,0.203180,0.174222,0.188466
0,7371,700,800,8,0.023646,7116.284899,6573.638247,7492.329736,6918.422303,5944,0.401898,0.202473,0.330816,0.257254
0,7371,800,900,3,0.009922,1823.664313,2220.428978,2264.683700,2160.359727,2494,0.268779,0.109692,0.205320,0.148149
0,7371,900,1000,7,0.026129,6110.730003,6845.326091,7035.161981,6803.902096,6568,0.189538,0.144750,0.135519,0.130018


In [83]:
df4_small = df4[df4.salesForecast_y<=100]
sales_threshold = np.arange(0, 100, 5)
tmp_list = []
for i in range(len(sales_threshold)):
    if i != len(sales_threshold)-1:
        tmp = df4_small[(sales_threshold[i]<df4_small.salesForecast_y)&(df4_small.salesForecast_y <= sales_threshold[i+1])]
        print('range in ',sales_threshold[i], sales_threshold[i+1], 'shape:', tmp.shape)
        tmp_oneline_df = pd.DataFrame()
        tmp_oneline_df['cate3'] = [cate3]
        tmp_oneline_df['range_min'] = [sales_threshold[i]]
        tmp_oneline_df['range_max'] = [sales_threshold[i+1]]
        tmp_oneline_df['count_num'] = [tmp.shape[0]]
        tmp_oneline_df['sales_ratio_total'] = [float(tmp.salesForecast_y.sum()) / df4_small.salesForecast_y.sum()]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['online_pred_sum'] = [tmp.ofdsales.sum()]
        tmp_oneline_df['ypred_raw_sum'] = [tmp.ypred_raw.sum()]
        tmp_oneline_df['model_2_ypred_sum'] = [tmp.model_2_ypred.sum()]
        tmp_oneline_df['ensemble_sum'] = [tmp.final_ensemble.sum()]
        tmp_oneline_df['actual_sales_sum'] = [tmp.salesForecast_y.sum()]
        tmp_oneline_df['online_pred_mape'] = [np.sum(np.abs(tmp.ofdsales - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ypred_raw_mape'] = [np.sum(np.abs(tmp.ypred_raw - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['model_2_ypred_mape'] = [np.sum(np.abs(tmp.model_2_ypred - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ensemble_mape'] = [np.sum(np.abs(tmp.final_ensemble - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
    else:
        tmp = df4_small[(sales_threshold[i]<df4_small.salesForecast_y)]
        print('larger than',sales_threshold[i], 'shape:', tmp.shape)
        tmp_oneline_df = pd.DataFrame()
        tmp_oneline_df['cate3'] = [cate3]
        tmp_oneline_df['range_min'] = [sales_threshold[i]]
        tmp_oneline_df['range_max'] = [-999]
        tmp_oneline_df['count_num'] = [tmp.shape[0]]
        tmp_oneline_df['sales_ratio_total'] = [float(tmp.salesForecast_y.sum()) / df4_small.salesForecast_y.sum()]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['online_pred_sum'] = [tmp.ofdsales.sum()]
        tmp_oneline_df['ypred_raw_sum'] = [tmp.ypred_raw.sum()]
        tmp_oneline_df['model_2_ypred_sum'] = [tmp.model_2_ypred.sum()]
        tmp_oneline_df['ensemble_sum'] = [tmp.final_ensemble.sum()]
        tmp_oneline_df['actual_sales_sum'] = [tmp.salesForecast_y.sum()]
        tmp_oneline_df['online_pred_mape'] = [np.sum(np.abs(tmp.ofdsales - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ypred_raw_mape'] = [np.sum(np.abs(tmp.ypred_raw - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['model_2_ypred_mape'] = [np.sum(np.abs(tmp.model_2_ypred - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
        tmp_oneline_df['ensemble_mape'] = [np.sum(np.abs(tmp.final_ensemble - tmp.salesForecast_y)) / np.sum(tmp.salesForecast_y)]\
                                                if np.sum(tmp.salesForecast_y) != 0 else np.nan
    tmp_list.append(tmp_oneline_df)
        
stat_df_small = pd.concat(tmp_list)          


range in  0 5 shape: (6182, 13)
range in  5 10 shape: (1577, 13)
range in  10 15 shape: (777, 13)
range in  15 20 shape: (467, 13)
range in  20 25 shape: (293, 13)
range in  25 30 shape: (223, 13)
range in  30 35 shape: (169, 13)
range in  35 40 shape: (114, 13)
range in  40 45 shape: (100, 13)
range in  45 50 shape: (88, 13)
range in  50 55 shape: (67, 13)
range in  55 60 shape: (66, 13)
range in  60 65 shape: (61, 13)
range in  65 70 shape: (55, 13)
range in  70 75 shape: (47, 13)
range in  75 80 shape: (40, 13)
range in  80 85 shape: (35, 13)
range in  85 90 shape: (31, 13)
range in  90 95 shape: (32, 13)
larger than 95 shape: (31, 13)


In [88]:
stat_df_small.online_pred_mape < stat_df_small.ypred_raw_mape

0    False
0     True
0    False
0     True
0    False
0    False
0    False
0    False
0    False
0     True
0    False
0    False
0    False
0     True
0    False
0     True
0    False
0    False
0    False
0    False
dtype: bool

In [86]:
stat_df_small

,cate3,range_min,range_max,count_num,sales_ratio_total,online_pred_sum,ypred_raw_sum,model_2_ypred_sum,ensemble_sum,actual_sales_sum,online_pred_mape,ypred_raw_mape,model_2_ypred_mape,ensemble_mape
0,7371,0,5,6182,0.125218,19323.601709,15616.029574,20231.423999,16904.516708,13576,0.986254,0.835588,1.027943,0.886493
0,7371,5,10,1577,0.112702,11072.551136,10420.279954,12397.919080,10936.863013,12219,0.622287,0.631394,0.667095,0.622597
0,7371,10,15,777,0.090916,8768.372034,8313.454976,9880.247636,8709.446561,9857,0.608954,0.589993,0.641761,0.587436
0,7371,15,20,467,0.076767,7226.541603,7421.914077,8788.756342,7660.499901,8323,0.552168,0.566652,0.616311,0.550288
0,7371,20,25,293,0.061640,5352.528179,5403.387790,6593.215672,5662.118877,6683,0.582128,0.530454,0.590162,0.522150
0,7371,25,30,223,0.057241,4786.308853,4995.809162,5851.560197,5116.636349,6206,0.489134,0.469024,0.493505,0.448382
0,7371,30,35,169,0.051347,4483.380804,4758.325718,5438.267567,4864.916147,5567,0.520442,0.493294,0.509439,0.475594
0,7371,35,40,114,0.039864,3547.464882,3984.073907,4563.266561,4059.979697,4322,0.480455,0.471631,0.492581,0.446760
0,7371,40,45,100,0.039375,2780.789181,3239.325731,3509.459574,3247.323004,4269,0.416798,0.387441,0.346870,0.355524
0,7371,45,50,88,0.039154,3253.949343,3535.188591,3902.438649,3547.537717,4245,0.372337,0.431179,0.429574,0.409045
